# The ResNet architecture, from identity block to the whole structure

* The first part of building the ResNet architecture is the identity block,

Which will be the base for the whole structure.

In [23]:
#!/usr/bin/env python3
"""
Makes the ResNet model identity block.
"""


import tensorflow.keras as keras


def identity_block(A_prev, filters):
    """
    Identity block function.
    
    :param A_prev: output of the previous layer.
    
    :param filters: number of filters in tuple form carrying:
    F11 - filter for 1x1 convolution
    F3 - filter size for 3x3 convolution
    F22 - filter for second 1z1 convolution
    
    All convolutions will be followed by Batch normalization, and ReLU activation.
    
    All weights will use normal initialization.
    
    Returns: Activated output of the block.
    """
    F11, F3, F12 = filters
    initializer = keras.initializers.he_normal()
    activation = keras.activations.relu
    
    layers = keras.layers
    
    Conv_1x1 = layers.Conv2D(
        F11,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,   
    )(A_prev)
    Batch_1x1 = layers.BatchNormalization(axis=3)(Conv_1x1)
    ReLU_1x1 = layers.Activation(activation)(Batch_1x1)
    
    Conv_3x3 = layers.Conv2D(
        F3,
        (3, 3),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_1x1)
    Batch_3x3 = layers.BatchNormalization(axis=3)(Conv_3x3)
    ReLU_3x3 = layers.Activation(activation)(Batch_3x3)
    
    Conv_1x1_2 = layers.Conv2D(
        F12,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_3x3)
    Batch_1x1_2 = layers.BatchNormalization(axis=3)(Conv_1x1_2)
    
    pre_output = layers.Add()([Batch_1x1_2, A_prev])
    
    output = layers.Activation(activation)(pre_output)
    
    return output


In [24]:
# Main function for Identity Block

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 256))
    Y = identity_block(X, [64, 64, 256])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 224, 224,  │     16,448 │ input_layer_10[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_40[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 224, 224,  │     36,928 │ activation_18[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_41[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 224, 224,  │     16,640 │ activation_19[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │      1,024 │ conv2d_42[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 224, 224,  │          0 │ batch_normalizat… │
│                     │ 256)              │            │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 224, 224,  │          0 │ add_6[0][0]       │
│ (Activation)        │ 256)              │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 71,552 (279.50 KB)

 Trainable params: 70,784 (276.50 KB)

 Non-trainable params: 768 (3.00 KB)

# Resnet Projection Block
After Completion of the identity_block, we can move on to the projection_block.

The projection block is used when the input and output dimensions are different for eachother.
* The way it works is this:

> It includes a convolutional layer with a stride of (2, 2), in order to downsample the input
> 
> And a convolutional layer with a (1, 1) filter size to change the depth of the input to match the output. 

In [25]:
#!/usr/bin/env python3
"""
Makes the Projection blocks of the ResNet Architecture
"""


import tensorflow.keras as keras


def projection_block(A_prev, filters, s=2):
    """
    Projection block function.
    
    :param A_prev: output of the previous layer.
    
    :param filters: number of filters in tuple form carrying:
    F11 - filter for 1x1 convolution
    F3 - filter size for 3x3 convolution
    F12 -filter for second 1z1 convolution as well as the shortcut convolution
    
    s- stride of the main and shortcut convolution
    
    All convolutions will be followed by Batch normalization,
    and ReLU activation.
    
    All weights will use he_normal initialization.
    Seed of he_normal initialization will be set to 0
    
    Returns: Activated output of the block.
    """
    F11, F3, F12 = filters
    initializer = keras.initializers.he_normal(seed=0)
    activation = keras.activations.relu
    layers = keras.layers
    batch = layers.BatchNormalization
    
    Conv_1x1 = layers.Conv2D(
        F11,
        (1, 1),
        strides=s,
        padding='same',
        kernel_initializer=initializer,
    )(A_prev)
    Batch_1x1 = batch(axis=3)(Conv_1x1)
    ReLU_1x1 = layers.Activation(activation)(Batch_1x1)
    
    Conv_3x3 = layers.Conv2D(
        F3,
        (3, 3),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_1x1)
    Batch_3x3 = batch(axis=3)(Conv_3x3)
    ReLU_3x3 = layers.Activation(activation)(Batch_3x3)
    
    Conv_1x1_2 = layers.Conv2D(
        F12,
        (1, 1),
        padding='same',
        kernel_initializer=initializer,
    )(ReLU_3x3)
    Batch_1x1_2 = batch(axis=3)(Conv_1x1_2)
    
    skip_layer = layers.Conv2D(
        F12,
        (1, 1),
        strides=s,
        padding='same',
        kernel_initializer=initializer,
    )(A_prev)
    skip_Batch = batch(axis=3)(skip_layer)
    
    pre_output = layers.Add()([Batch_1x1_2, skip_Batch])
    
    output = layers.Activation(activation)(pre_output)
    
    return output


In [26]:
# Main function of Projection Block

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 3))
    Y = projection_block(X, [64, 64, 256])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 112, 112,  │        256 │ input_layer_11[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_43[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_21       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 112, 112,  │     36,928 │ activation_21[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_44[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_22       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 112, 112,  │     16,640 │ activation_22[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 112, 112,  │      1,024 │ input_layer_11[0… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_45[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_46[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_23       │ (None, 112, 112,  │          0 │ add_7[0][0]       │
│ (Activation)        │ 256)              │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,408 (224.25 KB)

 Trainable params: 56,128 (219.25 KB)

 Non-trainable params: 1,280 (5.00 KB)

# With the Founding Blocks complete, All that is left is the model

* Using the modules built we will make the ResNet Architecture.

In [27]:
#!/usr/bin/env python3
"""
Makes the ResNet Architecture
"""

from tensorflow import keras
"""
identity_block = __import__('2-identity_block').identity_block
projection_block = __import__('3-projection_block').projection_block
"""


def resnet50():
    """
    ResNet50 architecture.
    
    The input image is 224x224 RGB.
    
    All convolutions will be followed by Batch normalization,
    and ReLU activation.
    
    All weights will use he_normal initialization.
    Seed of he_normal initialization will be set to 0
    
    Returns: Keras Model
    """
    init = keras.initializers.he_normal()
    activation = keras.activations.relu
    input = keras.Input(shape=(224, 224, 3))
    layers = keras.layers
    
    start = layers.Conv2D(
        64,
        (7, 7),
        strides=(2, 2),
        padding='same',
        kernel_initializer=init,
    )(input)
    start_batch = layers.BatchNormalization(axis=3)(start)
    start_relu = layers.Activation(activation)(start_batch)
    
    max_pool_1 = (layers.MaxPool2D(
        pool_size=(3, 3),
        strides=(2, 2),
        padding='same',
    )(start_relu))
    
    project_1 = projection_block(start_relu, [64, 64, 256], s=1)
    identity_1 = identity_block(
        project_1,
        [64, 64, 256],
    )
    identity_2 = identity_block(
        identity_1,
        [64, 64, 256],
    )
    
    project_2 = projection_block(identity_2, [128, 128, 512], s=2)
    identity_2_1 = identity_block(
        project_2,
        [128, 128, 512]
    )
    identity_2_2 = identity_block(
        identity_2_1,
        [128, 128, 512],
    )
    identity_2_3 = identity_block(
        identity_2_2,
        [128, 128, 512],
    )
    
    project_3 = projection_block(identity_2_3, [256, 256, 1024], s=2)
    identity_3_1 = identity_block(
        project_3,
        [256, 256, 1024],
    )
    identity_3_2 = identity_block(
        identity_3_1,
        [256, 256, 1024],
    )
    identity_3_3 = identity_block(
        identity_3_2,
        [256, 256, 1024],
    )
    identity_3_4 = identity_block(
        identity_3_3,
        [256, 256, 1024],
    )
    identity_3_5 = identity_block(
        identity_3_4,
        [256, 256, 1024],
    )
    
    project_4 = projection_block(identity_3_5, [512, 512, 2048], s=2)
    identity_4_1 = identity_block(
        project_4,
        [512, 512, 2048],
    )
    identity_4_2 = identity_block(
        identity_4_1,
        [512, 512, 2048],
    )
    
    average_pooling = layers.AveragePooling2D(
        pool_size=(7, 7),
        strides=(1, 1),
        padding="valid"
    )(identity_4_2)
    
    output = layers.Dense(
        1000,
        activation="softmax",
        kernel_initializer=init,
    )(average_pooling)
    
    model = keras.models.Model(inputs=input, outputs=output)
    
    return model
    

In [28]:
# Main function for Resnet50

if __name__ == '__main__':
    model = resnet50()
    model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 112, 112,  │      9,472 │ input_layer_12[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_47[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_24       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 112, 112,  │      4,160 │ activation_24[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_48[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_25       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 112, 112,  │     36,928 │ activation_25[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_49[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_26       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 112, 112,  │     16,640 │ activation_26[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 112, 112,  │     16,640 │ activation_24[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_50[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │      1,024 │ conv2d_51[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_27       │ (None, 112, 112,  │          0 │ add_8[0][0]       │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 112, 112,  │     16,448 │ activation_27[0]

 Total params: 25,636,712 (97.80 MB)

 Trainable params: 25,583,592 (97.59 MB)

 Non-trainable params: 53,120 (207.50 KB)